In [29]:
import sqlite3
import random
import pandas as pd
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Create in-memory SQLite connection
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

In [31]:
## Sample data:
# Parameters
start_date = datetime(2024, 1, 1)
end_date = datetime(2024, 12, 31)
num_rows = 1000

# Generate random sales data
sales_data_large = []
for order_id in range(1, num_rows + 1):
    # Random date between start_date and end_date
    random_days = random.randint(0, (end_date - start_date).days)
    order_date = start_date + timedelta(days=random_days)
    
    # Random sales amount between 50 and 500
    amount = round(random.uniform(50, 500), 2)
    
    sales_data_large.append((order_id, order_date.strftime('%Y-%m-%d'), amount))

# Convert to DataFrame for preview
df_large = pd.DataFrame(sales_data_large, columns=['order_id', 'order_date', 'amount'])
df_large.head()

,order_id,order_date,amount
0,1,2024-01-23,191.30
1,2,2024-04-12,246.16
2,3,2024-11-30,216.91
3,4,2024-08-11,311.44
4,5,2024-01-25,293.64


In [32]:
# Load DataFrame into SQLite table
df_large.to_sql("sales", conn, index=False, if_exists="replace")

1000

#### 1. Calculate Month over Month sales

In [44]:
pd.read_sql_query("""
WITH monthly_sales AS (
    SELECT
    DATE(strftime('%Y-%m-01', order_date)) AS month_start,
    SUM(amount) AS total_sales
    FROM sales
    GROUP BY strftime('%Y-%m', order_date)
    )
    SELECT
    month_start,
    total_sales,
    prev_month_sales,
    total_sales - prev_month_sales AS mom_change,
    concat(coalesce(ROUND((total_sales - prev_month_sales) * 100.0 / prev_month_sales, 2), 0), '%') AS mom_change_pct
    FROM (
        SELECT
        month_start,
        total_sales,
        LAG(total_sales) OVER (ORDER BY month_start) AS prev_month_sales
        FROM monthly_sales
        ) t
    ORDER BY month_start;
""", conn)

,month_start,total_sales,prev_month_sales,mom_change,mom_change_pct
0,2024-01-01,20696.06,NaN,NaN,0%
1,2024-02-01,25735.94,20696.06,5039.88,24.35%
2,2024-03-01,16761.96,25735.94,-8973.98,-34.87%
3,2024-04-01,26584.18,16761.96,9822.22,58.6%
4,2024-05-01,20638.95,26584.18,-5945.23,-22.36%
5,2024-06-01,19235.37,20638.95,-1403.58,-6.8%
6,2024-07-01,26979.78,19235.37,7744.41,40.26%
7,2024-08-01,24062.40,26979.78,-2917.38,-10.81%
8,2024-09-01,21330.89,24062.40,-2731.51,-11.35%
9,2024-10-01,28427.32,21330.89,7096.43,33.27%
